In [1]:
from PIL import Image as PILImage
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['image.origin'] = 'lower'  # we want to show images, not matrices, so we set the origin to the lower-left
plt.matplotlib.style.use('dark_background') 

from astropy import units as u
from astropy.modeling.polynomial import Polynomial1D
from astropy.modeling.models import Gaussian1D, Linear1D
from astropy.modeling.fitting import LinearLSQFitter, LevMarLSQFitter
from IPython.display import Image
from astropy.table import Table

# astroquery provides an interface to the NIST atomic line database
from astroquery.nist import Nist
from astropy.io import fits

import glob
import os
from astropy.visualization import ZScaleInterval

from pathlib import Path

import lacosmic
from astropy.nddata import CCDData
import ccdproc

In [17]:
calibrated_data = Path('.', 'reduced_data')
calibrated_data.mkdir(exist_ok=True)
raw_darks = ccdproc.ImageFileCollection('../soar/darks')
darks = raw_darks.files_filtered(include_path=True)
raw_darks.summary['file', 'detsize', 'trimsec', 'ccdsize']

file,detsize,trimsec,ccdsize
str14,str15,str16,str15
0026_dark.fits,"[1:4096,1:4096]","[51:4110,2:1896]","[1:4096,1:4096]"
0027_dark.fits,"[1:4096,1:4096]","[51:4110,2:1896]","[1:4096,1:4096]"
0028_dark.fits,"[1:4096,1:4096]","[51:4110,2:1896]","[1:4096,1:4096]"
0029_dark.fits,"[1:4096,1:4096]","[51:4110,2:1896]","[1:4096,1:4096]"
0030_dark.fits,"[1:4096,1:4096]","[51:4110,2:1896]","[1:4096,1:4096]"


In [21]:
for file in darks:
    print(file[14:18])

0026
0027
0028
0029
0030


In [ ]:
def overscan_and_trim(filelist):
    for num, file in enumerate(filelist):
        filename=file[14:18]
        file_data = CCDData.read(file, unit='adu')
        overscan_subtracted = ccdproc.subtract_overscan(file_data, overscan=file_data[:1, :], median=True)
        trimmed = ccdproc.trim_image(overscan_subtracted[1:, :])
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

        show_image(file_data.data, cmap='gray', ax=ax1, fig=fig)
        ax1.set_title(f'Raw {name}')
        show_image(trimmed.data, cmap='gray', ax=ax2, fig=fig)
        ax2.set_title(f'{name}, overscan subtracted and trimmed')
        plt.show()
        plt.close()
        trimmed.write(calibrated_data / filename, overwrite='True')

['../soar/science/0112_sci.fits',
 '../soar/science/0113_sci.fits',
 '../soar/science/0114_sci.fits',
 '../soar/science/0115_sci.fits']

files